In [1]:
import pandas as pd
from datasets import load_dataset

# Search for a suitable dataset on Hugging Face
# We will start with a common toxicity dataset, for example, 'hate_speech_offensive'
try:
    dataset = load_dataset('hate_speech_offensive')
    print("Dataset 'hate_speech_offensive' loaded successfully.")
    print(dataset)
except Exception as e:
    print(f"Could not load 'hate_speech_offensive': {e}")
    print("Trying alternative datasets or outlining a manual labeling strategy.")

# If 'hate_speech_offensive' is not ideal or not found, we might explore others.
# For this demonstration, we will proceed assuming 'hate_speech_offensive' is a good starting point.
# Let's inspect the dataset to understand its structure and labels.

# Convert to pandas DataFrame for easier inspection (optional, but good for exploration)
# We'll take the 'train' split if available
if 'train' in dataset:
    df = dataset['train'].to_pandas()
    print("\nFirst 5 rows of the dataset:")
    print(df.head())
    print("\nDataset columns:")
    print(df.columns)
    print("\nValue counts for 'class' column (or similar label column):")
    if 'class' in df.columns:
        print(df['class'].value_counts())
    elif 'label' in df.columns:
        print(df['label'].value_counts())
    else:
        print("No 'class' or 'label' column found. Please inspect the dataset manually.")
else:
    print("No 'train' split found in the dataset. Please inspect the dataset manually.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24783 [00:00<?, ? examples/s]

Dataset 'hate_speech_offensive' loaded successfully.
DatasetDict({
    train: Dataset({
        features: ['count', 'hate_speech_count', 'offensive_language_count', 'neither_count', 'class', 'tweet'],
        num_rows: 24783
    })
})

First 5 rows of the dataset:
   count  hate_speech_count  offensive_language_count  neither_count  class  \
0      3                  0                         0              3      2   
1      3                  0                         3              0      1   
2      3                  0                         3              0      1   
3      3                  0                         2              1      1   
4      6                  0                         6              0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_bas

In [2]:
import pandas as pd
from datasets import load_dataset

# Assuming the dataset 'hate_speech_offensive' has been loaded into `dataset` and converted to `df`
# If starting fresh or rerunning, ensure dataset and df are available
# try:
#     dataset = load_dataset('hate_speech_offensive')
#     df = dataset['train'].to_pandas()
# except NameError:
#     print("Dataset or DataFrame not found. Please run the previous cell first.")

# Map the 'class' column to descriptive labels
# According to the dataset documentation:
# 0: hate speech
# 1: offensive language
# 2: neither
class_mapping = {
    0: 'Hate Speech',
    1: 'Offensive Language',
    2: 'Neither'
}
df['label'] = df['class'].map(class_mapping)

# Rename the 'tweet' column to 'text' for consistency
df = df.rename(columns={'tweet': 'text'})

# Drop original 'class' column and other count columns as they are no longer needed for classification
df = df.drop(columns=['class', 'count', 'hate_speech_count', 'offensive_language_count', 'neither_count'])

print("Dataset after label mapping, column renaming, and dropping unnecessary columns:")
print(df.head())
print("\nValue counts for the new 'label' column:")
print(df['label'].value_counts())
print("\nUpdated Dataset columns:")
print(df.columns)

Dataset after label mapping, column renaming, and dropping unnecessary columns:
                                                text               label
0  !!! RT @mayasolovely: As a woman you shouldn't...             Neither
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  Offensive Language
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  Offensive Language
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  Offensive Language
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  Offensive Language

Value counts for the new 'label' column:
label
Offensive Language    19190
Neither                4163
Hate Speech            1430
Name: count, dtype: int64

Updated Dataset columns:
Index(['text', 'label'], dtype='object')


In [3]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

# 1. Map string labels to numerical IDs
# Re-using the class_mapping from before, but in reverse for label2id
label_to_id = {label: idx for idx, label in class_mapping.items()}
df['label_id'] = df['label'].map(label_to_id)

# 2. Create id2label and label2id dictionaries
id2label = {idx: label for label, idx in label_to_id.items()}
# Ensure id2label is sorted by key for consistency if needed, but not strictly necessary here

print("Label to ID mapping:")
print(label_to_id)
print("ID to Label mapping:")
print(id2label)

# 3. Split the DataFrame into training, validation, and test sets
# First, split into train (80%) and temp (20%)
df_train, df_temp = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label_id'])

# Then, split temp (20%) into validation (10%) and test (10%) - relative to original df size
# This means splitting df_temp into two equal halves (0.5 of df_temp = 0.1 of original df)
df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=42, stratify=df_temp['label_id'])

print(f"\nTraining set size: {len(df_train)}")
print(f"Validation set size: {len(df_val)}")
print(f"Test set size: {len(df_test)}")

# Verify stratification
print("\nTraining set label distribution:")
print(df_train['label'].value_counts(normalize=True))
print("\nValidation set label distribution:")
print(df_val['label'].value_counts(normalize=True))
print("\nTest set label distribution:")
print(df_test['label'].value_counts(normalize=True))

# 4. Convert pandas DataFrames to datasets.Dataset objects and store in a DatasetDict
hf_dataset = DatasetDict({
    'train': Dataset.from_pandas(df_train, preserve_index=False),
    'validation': Dataset.from_pandas(df_val, preserve_index=False),
    'test': Dataset.from_pandas(df_test, preserve_index=False)
})

print("\nHugging Face DatasetDict structure:")
print(hf_dataset)

# Display a sample from the training set to show the format
print("\nSample from training set (first 5 examples):")
print(hf_dataset['train'].to_pandas().head())

Label to ID mapping:
{'Hate Speech': 0, 'Offensive Language': 1, 'Neither': 2}
ID to Label mapping:
{0: 'Hate Speech', 1: 'Offensive Language', 2: 'Neither'}

Training set size: 19826
Validation set size: 2478
Test set size: 2479

Training set label distribution:
label
Offensive Language    0.774337
Neither               0.167961
Hate Speech           0.057702
Name: proportion, dtype: float64

Validation set label distribution:
label
Offensive Language    0.774415
Neither               0.167877
Hate Speech           0.057708
Name: proportion, dtype: float64

Test set label distribution:
label
Offensive Language    0.774102
Neither               0.168213
Hate Speech           0.057685
Name: proportion, dtype: float64

Hugging Face DatasetDict structure:
DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_id'],
        num_rows: 19826
    })
    validation: Dataset({
        features: ['text', 'label', 'label_id'],
        num_rows: 2478
    })
    test: Dataset

In [4]:
from transformers import AutoTokenizer

# Define the model checkpoint (e.g., a BERT-like model for text classification)
# 'bert-base-uncased' is a common choice for its balance of performance and size
model_checkpoint = "bert-base-uncased"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Define a tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

# Apply the tokenization function to the entire dataset
# The `batched=True` argument processes multiple examples at once, which is faster.
# The `num_proc` argument can be used for multiprocessing if resources allow.
# The `remove_columns` argument removes the original 'text' and 'label' columns after tokenization
# as they are no longer needed in their raw form for model input.
hf_tokenized_dataset = hf_dataset.map(tokenize_function, batched=True, remove_columns=["text", "label"])

# Rename the 'label_id' column to 'labels' to match the expected format for Hugging Face Trainers
hf_tokenized_dataset = hf_tokenized_dataset.rename_column("label_id", "labels")

print("Tokenization complete. Updated DatasetDict structure:")
print(hf_tokenized_dataset)

print("\nSample from tokenized training set (first example):")
print(hf_tokenized_dataset["train"][0])

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/19826 [00:00<?, ? examples/s]

Map:   0%|          | 0/2478 [00:00<?, ? examples/s]

Map:   0%|          | 0/2479 [00:00<?, ? examples/s]

Tokenization complete. Updated DatasetDict structure:
DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 19826
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2478
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2479
    })
})

Sample from tokenized training set (first example):
{'labels': 1, 'input_ids': [101, 3331, 10413, 2003, 1037, 7570, 2063, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [5]:
from transformers import AutoModelForSequenceClassification

# Load a pre-trained model for sequence classification
# Use the same model_checkpoint as for the tokenizer
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
)

print("Pre-trained model loaded successfully:")
print(model)


NameError: name 'label2id' is not defined

In [6]:
from transformers import AutoModelForSequenceClassification

# Load a pre-trained model for sequence classification
# Use the same model_checkpoint as for the tokenizer
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label_to_id
)

print("Pre-trained model loaded successfully:")
print(model)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: bert-base-uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
classifier.weight                          | MISSING    | 
classifier.bias                            | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Pre-trained model loaded successfully:
BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (L

In [7]:
from transformers import TrainingArguments, Trainer

# Define TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir="./logs",            # directory for storing logs
    logging_steps=10,                # log every N updates steps
    evaluation_strategy="epoch",     # evaluate each epoch
    save_strategy="epoch",           # save checkpoint each epoch
    load_best_model_at_end=True,     # load the best model when training ends
    metric_for_best_model="f1",      # metric to use to compare models (e.g., f1, accuracy)
    seed=42,                         # random seed for reproducibility
)

print("TrainingArguments defined successfully:")
print(training_args)


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [8]:
from transformers import TrainingArguments, Trainer
from transformers.training_args import IntervalStrategy

# Define TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir="./logs",            # directory for storing logs
    logging_steps=10,                # log every N updates steps
    evaluation_strategy=IntervalStrategy.EPOCH,     # evaluate each epoch
    save_strategy=IntervalStrategy.EPOCH,           # save checkpoint each epoch
    load_best_model_at_end=True,     # load the best model when training ends
    metric_for_best_model="f1",      # metric to use to compare models (e.g., f1, accuracy)
    seed=42,                         # random seed for reproducibility
)

print("TrainingArguments defined successfully:")
print(training_args)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [9]:
from transformers import TrainingArguments, Trainer
from transformers.training_args import IntervalStrategy

# Define TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir="./logs",            # directory for storing logs
    logging_steps=10,                # log every N updates steps
    eval_strategy=IntervalStrategy.EPOCH,     # Corrected from 'evaluation_strategy'
    save_strategy=IntervalStrategy.EPOCH,           # 'save_strategy' is generally consistent
    load_best_model_at_end=True,     # load the best model when training ends
    metric_for_best_model="f1",      # metric to use to compare models (e.g., f1, accuracy)
    seed=42,                         # random seed for reproducibility
)

print("TrainingArguments defined successfully:")
print(training_args)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


TrainingArguments defined successfully:
TrainingArguments(
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=True,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
enable_jit_checkpoint=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStr

In [10]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

print("compute_metrics function defined successfully.")


compute_metrics function defined successfully.


In [11]:
from transformers import Trainer, DataCollatorWithPadding

# Initialize the Data Collator. This will dynamically pad batches of examples
# to the longest length in the batch, rather than padding all examples to the
# maximum length supported by the model.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_tokenized_dataset["train"],
    eval_dataset=hf_tokenized_dataset["validation"],
    tokenizer=tokenizer, # Pass tokenizer to trainer for data collation and preprocessing
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("Trainer initialized successfully.")

TypeError: Trainer.__init__() got an unexpected keyword argument 'tokenizer'

In [12]:
from transformers import Trainer, DataCollatorWithPadding

# Initialize the Data Collator. This will dynamically pad batches of examples
# to the longest length in the batch, rather than padding all examples to the
# maximum length supported by the model.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_tokenized_dataset["train"],
    eval_dataset=hf_tokenized_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("Trainer initialized successfully.")

Trainer initialized successfully.


In [13]:
print("Starting model training...")
trainer.train()

print("Training complete. Saving the fine-tuned model and tokenizer...")
# Define the output directory for the fine-tuned model
output_model_dir = "./fine_tuned_model"
trainer.save_model(output_model_dir)
tokenizer.save_pretrained(output_model_dir)

print(f"Fine-tuned model and tokenizer saved to: {output_model_dir}")

Starting model training...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.265427,0.240014,0.913640,0.914280,0.915145,0.913640
2,0.273200,0.233155,0.913236,0.900307,0.896922,0.913236
3,0.169532,0.274336,0.920500,0.917512,0.915581,0.920500


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.output.LayerNorm.weight', 'bert.encoder.layer.0.output.LayerNorm.bias', 'bert.encoder.layer.1.attention.output.LayerNorm.weight', 'bert.encoder.layer.1.attention.output.LayerNorm.bias', 'bert.encoder.layer.1.output.LayerNorm.weight', 'bert.encoder.layer.1.output.LayerNorm.bias', 'bert.encoder.layer.2.attention.output.LayerNorm.weight', 'bert.encoder.layer.2.attention.output.LayerNorm.bias', 'bert.encoder.layer.2.output.LayerNorm.weight', 'bert.encoder.layer.2.output.LayerNorm.bias', 'bert.encoder.layer.3.attention.output.LayerNorm.weight', 'bert.encoder.layer.3.attention.output.LayerNorm.bias', 'bert.encoder.layer.3.output.LayerNorm.weight', 'bert.encoder.layer.3.output.LayerNorm.bias', 'bert.encoder.layer.4.attention.output.La

Training complete. Saving the fine-tuned model and tokenizer...


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Fine-tuned model and tokenizer saved to: ./fine_tuned_model


In [14]:
print("Evaluating the fine-tuned model on the test set...")
results = trainer.evaluate(hf_tokenized_dataset["test"])

print("Evaluation results:")
for key, value in results.items():
    print(f"{key}: {value}")

Evaluating the fine-tuned model on the test set...


Evaluation results:
eval_loss: 0.27534621953964233
eval_accuracy: 0.9197256958450988
eval_f1: 0.9155031898496764
eval_precision: 0.9129864486665151
eval_recall: 0.9197256958450988
eval_runtime: 10.8661
eval_samples_per_second: 228.14
eval_steps_per_second: 14.265
epoch: 3.0


In [15]:
from transformers import pipeline

# Load the fine-tuned model and tokenizer
loaded_tokenizer = AutoTokenizer.from_pretrained(output_model_dir)
loaded_model = AutoModelForSequenceClassification.from_pretrained(output_model_dir)

# Create a text classification pipeline using the loaded model and tokenizer
classifier = pipeline(
    "text-classification",
    model=loaded_model,
    tokenizer=loaded_tokenizer
)

print("Model and tokenizer loaded, and classification pipeline created successfully.")

# Example usage with user input
user_inputs = [
    "I love spending time with my family and friends.",
    "You are so stupid and ugly, I hate you.",
    "What a beautiful day! The sun is shining.",
    "I am going to punch you in the face.",
    "This is a neutral sentence about a common topic."
]

print("\nClassifying user inputs:")
for text in user_inputs:
    prediction = classifier(text)
    # The pipeline returns a list of dictionaries, take the first one
    label = prediction[0]['label']
    score = prediction[0]['score']
    print(f"Text: '{text}' -> Predicted Label: {label} (Score: {score:.4f})")


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Model and tokenizer loaded, and classification pipeline created successfully.

Classifying user inputs:
Text: 'I love spending time with my family and friends.' -> Predicted Label: Neither (Score: 0.9976)
Text: 'You are so stupid and ugly, I hate you.' -> Predicted Label: Hate Speech (Score: 0.9219)
Text: 'What a beautiful day! The sun is shining.' -> Predicted Label: Neither (Score: 0.9987)
Text: 'I am going to punch you in the face.' -> Predicted Label: Offensive Language (Score: 0.7696)
Text: 'This is a neutral sentence about a common topic.' -> Predicted Label: Neither (Score: 0.9979)
